<a href="https://colab.research.google.com/github/moazahmed-official/NLP/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("parve05/customer-review-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'customer-review-dataset' dataset.
Path to dataset files: /kaggle/input/customer-review-dataset


In [ ]:
!ls /root/.cache/kagglehub/datasets/parve05/customer-review-dataset/versions/1


redmi6.csv


In [ ]:
import pandas as pd

df = pd.read_csv("/root/.cache/kagglehub/datasets/parve05/customer-review-dataset/versions/1/redmi6.csv" , encoding='ISO-8859-1')

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

print("Columns in DataFrame:", df.columns.tolist())

def findTextColumn(columns):
    lowerCols = [col.lower() for col in columns]
    keywords = ['review', 'comments', 'comment', 'text', 'review text', 'reviewstext']
    for keyword in keywords:
        for i, col in enumerate(lowerCols):
            if keyword in col:
                return columns[i]
    objCols = [col for col in columns if df[col].dtype == 'object']
    if objCols:
        avgLen = {col: df[col].dropna().astype(str).map(len).mean() for col in objCols}
        return max(avgLen, key=avgLen.get)
    return None

textColumn = findTextColumn(df.columns.tolist())

if textColumn is None:
    raise KeyError("Couldn't find a text column automatically. Please pass it manually (e.g. 'Comments').")

print("Using text column:", textColumn)

df = df.copy()
df.rename(columns={textColumn: 'reviewText'}, inplace=True)

def preprocessText(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"http\S+|www\.\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s']", " ", text)
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text

df['cleanedText'] = df['reviewText'].apply(preprocessText)

stopWords = set(stopwords.words('english'))

def removeStopwords(tokens):
    return [t for t in tokens if t not in stopWords and len(t) > 1]

df['tokens'] = df['cleanedText'].apply(lambda x: nltk.word_tokenize(x) if x else [])
df['tokensNoStop'] = df['tokens'].apply(removeStopwords)

stemmer = PorterStemmer()
df['stemmedTokens'] = df['tokensNoStop'].apply(lambda toks: [stemmer.stem(t) for t in toks])

lemmatizer = WordNetLemmatizer()
df['lemmatizedTokens'] = df['tokensNoStop'].apply(lambda toks: [lemmatizer.lemmatize(t) for t in toks])

df.to_csv("cleanedProductReviews.csv", index=False, encoding='utf-8-sig')
print("✅ Cleaned dataset saved as 'cleanedProductReviews.csv'")

print("\nSample (first 5 rows):")
for i in range(min(5, len(df))):
    print("Original :", df['reviewText'].iloc[i])
    print("Cleaned  :", df['cleanedText'].iloc[i])
    print("Tokens   :", df['tokens'].iloc[i])
    print("NoStop   :", df['tokensNoStop'].iloc[i])
    print("Stemmed  :", df['stemmedTokens'].iloc[i])
    print("Lemmas   :", df['lemmatizedTokens'].iloc[i])
    print("-" * 60)


Columns in DataFrame: ['Review Title', 'Customer name', 'Rating', 'Date', 'Category', 'Comments', 'Useful']
Using text column: Review Title
✅ Cleaned dataset saved as 'cleanedProductReviews.csv'

Sample (first 5 rows):
Original : Another Midrange killer Smartphone by Xiaomi
Cleaned  : another midrange killer smartphone by xiaomi
Tokens   : ['another', 'midrange', 'killer', 'smartphone', 'by', 'xiaomi']
NoStop   : ['another', 'midrange', 'killer', 'smartphone', 'xiaomi']
Stemmed  : ['anoth', 'midrang', 'killer', 'smartphon', 'xiaomi']
Lemmas   : ['another', 'midrange', 'killer', 'smartphone', 'xiaomi']
------------------------------------------------------------
Original : vry small size mobile
Cleaned  : vry small size mobile
Tokens   : ['vry', 'small', 'size', 'mobile']
NoStop   : ['vry', 'small', 'size', 'mobile']
Stemmed  : ['vri', 'small', 'size', 'mobil']
Lemmas   : ['vry', 'small', 'size', 'mobile']
------------------------------------------------------------
Original : Full disp